In [1]:
import tensorflow as tf
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
import numpy as np

#load dataset and plot first 20 digits

digits = load_digits()

fig, axes = plt.subplots(4, 5, figsize=(10, 5),
                        subplot_kw={'xticks':(), 'yticks': ()})
for ax, img in zip(axes.ravel(), digits.images):
    ax.imshow(img)

In [2]:
#break data on train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    digits.data, digits.target,  random_state=42, shuffle=True)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
m, n = X_train.shape
print(X_train.shape)

(1347, 64)


In [3]:
#prepare log folder for tensoroard
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [4]:

#hyperparameters to fine-tune
n_neurons = 40
n_outputs = 10
learning_rate = 0.01


X = tf.placeholder(tf.float32, shape=(None, X_train.shape[1]), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

#first layer of the simple dnn
with tf.name_scope("first_layer"):
    init = np.random.randn(n, n_neurons) / np.sqrt(n / 2)
    W = tf.Variable(init, name="kernel", dtype=tf.float32)
    b = tf.Variable(tf.zeros([n_neurons]), name="bias", dtype=tf.float32)
    Z = tf.matmul(X, W) + b
    Z = tf.maximum(0.0, Z, name="activation")
    
#second layers
with tf.name_scope("last_layer"):
    init = np.random.randn(n_neurons, n_outputs) / np.sqrt(n_neurons / 2)
    W = tf.Variable(init, name="kernel", dtype=tf.float32)
    b = tf.Variable(tf.zeros([n_outputs]), name="bias", dtype=tf.float32)
    Z_out = tf.matmul(Z, W) + b
       
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=Z_out)
    loss = tf.reduce_mean(xentropy, name="loss")
    
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(Z_out, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
      
init = tf.global_variables_initializer()
saver = tf.train.Saver()

#prepare scalar for tensorboard
accuracy_summary = tf.summary.scalar('Accuracy', accuracy)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 1000
# batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X: X_train, y: y_train})
        summary_str_accuracy = accuracy_summary.eval(feed_dict={X: X_train, y: y_train})
        file_writer.add_summary(summary_str_accuracy, epoch)
        save_path = saver.save(sess, "./my_model_final.ckpt")

Instructions for updating:
Colocations handled automatically by placer.


In [19]:
#predict on test images and evaluate accuracy
correct = tf.nn.in_top_k(Z, y_test, 1)
cast = tf.cast(correct, tf.float32)
accuracy = tf.reduce_mean(cast)

with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    Z = Z_out.eval(feed_dict={X: X_test})
    y_pred = np.argmax(Z, axis=1)
    accuracy = accuracy.eval(feed_dict={X: X_test})
    
print("Test accuracy: {}".format(accuracy))

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
Test accuracy: 0.9510999917984009
